In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from meteo_imp.kalman.fastai import *
from meteo_imp.kalman.filter import *
from meteo_imp.utils import *
from meteo_imp.data import *

from fastai.tabular.learner import *
from fastai.learner import *
from fastai.callback.all import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import torch

In [ ]:
@cache_disk("full_hai")
def load_data():
    return read_fluxnet_csv(hai_path, None, num_dtype=np.float64)

hai = load_data()

shortcut for having randomly initialized and with correct type paramters

In [ ]:
model = KalmanFilterTester(dtype=torch.float64).filter

In [ ]:
# model = KalmanFilter(n_dim_state = hai.shape[1], n_dim_obs=hai.shape[1])

more sensible default to the obs_cov

In [ ]:
# model._set_constraint('obs_cov', model.obs_cov * 1e-1, train=True)

In [ ]:
model.obs_cov

tensor([[0.8357, 0.4744, 0.7844],
        [0.4744, 0.8175, 0.4352],
        [0.7844, 0.4352, 1.0748]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [ ]:
import pandas as pd

In [ ]:
class PosDefCallback(TrainEvalCallback):
    def __init__(self):
        self.info_posdef = pd.DataFrame()
    def after_batch(self):
        means, cov_batch = self.pred
        info = []
        for covs in cov_batch:
            for t, cov in enumerate(covs):
                info.append(check_posdef(cov.detach(), 'pred', error=False, t=t, n_iter = self.train_iter))
        self.info_posdef = pd.concat([self.info_posdef] + info)
        

In [ ]:
posdef_info = PosDefCallback()

In [ ]:
obs_cov_history = SaveParams('obs_cov')

In [ ]:
CollectDataCallback??

Init signature:
CollectDataCallback(
    *,
    after_create=None,
    before_fit=None,
    before_epoch=None,
    before_train=None,
    before_batch=None,
    after_pred=None,
    after_loss=None,
    before_backward=None,
    after_cancel_backward=None,
    after_backward=None,
    before_step=None,
    after_cancel_step=None,
    after_step=None,
    after_cancel_batch=None,
    after_batch=None,
    after_cancel_train=None,
    after_train=None,
    before_validate=None,
    after_cancel_validate=None,
    after_validate=None,
    after_cancel_epoch=None,
    after_epoch=None,
    after_cancel_fit=None,
    after_fit=None,
)
Source:        
class CollectDataCallback(Callback):
    "Collect all batches, along with `pred` and `loss`, into `self.data`. Mainly for testing"
    def before_fit(self): self.data = L()
    def after_batch(self): 
        self.data.append(self.learn.to_detach((self.xb,self.yb,self.pred,self.loss)))
File:           ~/anaconda3/envs/data-science/lib/python3.1

In [ ]:
saved_data = CollectDataCallback()

In [ ]:
dls = make_dataloader(hai[:5_000], 200, 10, bs=1) 

In [ ]:
learn = Learner(dls, model, loss_func=imp_ll_loss, cbs=[obs_cov_history, posdef_info])

In [ ]:
# learn.fit(5, 5e-3)

In [ ]:
learn.fit(10, 5e-2)

epoch,train_loss,valid_loss,time


RuntimeError: expected scalar type Double but found Float

In [ ]:
learn.recorder.plot_loss()

In [ ]:
len(obs_cov_history.params)

In [ ]:
obs_cov_history.params[-1]

In [ ]:
is_posdef2(obs_cov_history.params[-1])

In [ ]:
posdef_log

In [ ]:
posdef_info.info_posdef

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(posdef_info.info_posdef.t, posdef_info.info_posdef.is_sym)

In [ ]:
import altair as alt

In [ ]:
alt.data_transformers.enable('data_server')

In [ ]:
alt.Chart(posdef_info.info_posdef).mark_bar().encode(
    x = 't',
    y = 'count(sym_upto)',
    color='sym_upto:N'
)

In [ ]:
posdef_info.info_posdef.dtypes

In [ ]:
trained_state = learn.model.state_dict()

## Results

In [ ]:
from meteo_imp.data_preparation import MeteoDataTest
from meteo_imp.kalman.imputation import KalmanImputation

In [ ]:
def gap2res(var_sel, gap_len, gap_start, block_start=1000, block_end=1200):
    data = MeteoDataTest(hai.iloc[block_start:block_end, :]).add_gap(gap_len, var_sel, gap_start)
    imp = KalmanImputation(data.data)
    imp.model.filter.load_state_dict(trained_state)
    return imp.to_result(data.data_compl_tidy, var_names= data.data.columns, units=units, pred_all=True)

In [ ]:
data = MeteoDataTest(hai)

In [ ]:
var_sel = data.data.columns

In [ ]:
%time gap2res(var_sel, 10, 10)

In [ ]:
gaps = [2, 5, 7, 10, 20, 30, 50, 100]
gap_starts = [0, 30, 60, 90]

In [ ]:
from ipywidgets import interact_manual, IntSlider

In [ ]:
#| include: false
@interact_manual(TA=True,
                 SW_IN=True,
                 VPD=True,
                 gap_len=IntSlider(10, 1,100),
                 gap_start=IntSlider(30, 1,100),
                 block_start = IntSlider(1000, 0, 20_000, 1000),
                 block_len = IntSlider(200, 10, 1000, 10)
        )
def show_diff_gaps_res(TA, SW_IN, VPD, gap_len, gap_start, block_start, block_len):
    var_sel = []
    if TA: var_sel.append('TA')
    if SW_IN: var_sel.append('SW_IN')
    if VPD: var_sel.append('VPD')
    var_sel = (*var_sel,)
    gap2res(var_sel, gap_len, gap_start, block_start, block_start + block_len).display_results()
    